##### 파일 명세
- udrt_kakao_cdh와 udrt_kakao_cdh_updated2를 [사업장명]을 기준으로 새로운 컬렉션 생성
- 새로운 컬렉션(udrt_kakao_cdh_old&new_place_category) 필드=[uuid, category_name, place_name, mapped_category]

In [ ]:
import os
from dotenv import load_dotenv
from pymongo import MongoClient


In [ ]:
# 데이터베이스 연결
load_dotenv()
client = MongoClient(os.getenv('DB_ADR'),
          username=os.getenv('DB_USER'),
          password=os.getenv('DB_PASSWORD'),
          authSource=os.getenv('DB_AuthSource'),
          authMechanism=os.getenv('DB_AuthMechanism'))
db = client.get_database(os.getenv('DB_Collection'))

#컬렉션 불러오기
cl_udrt_kakao_cdh=db.get_collection("udrt_kakao_cdh")
cl_udrt_kakao_cdh_updated2=db.get_collection("udrt_kakao_cdh_updated2")

In [ ]:
# udrt_kakao_cdh_updated2의 값 딕셔너리로 변환
dict_udrt_kakao_cdh_updated2 = {doc["place_name"]: doc["mapped_category"] for doc in cl_udrt_kakao_cdh_updated2.find({}, {"place_name": 1, "mapped_category": 1, "_id": 0})}
print(f"udrt_kakao_cdh_updated2의 값 딕셔너리로 변환 완료.")

# udrt_kakao_cdh 가져와서 필드 병합 후 새로운 리스트 생성
new_documents = []
for doc in cl_udrt_kakao_cdh.find({}, {"uuid": 1, "category_name": 1, "place_name": 0, "_id": 0}):
    place_name = doc["place_name"]
    if place_name in dict_udrt_kakao_cdh_updated2:  # 필드1이 일치하는 경우
        new_documents.append({
            "uuid": doc["uuid"],
            "category_name":doc["category_name"],
            "place_name": doc["place_name"],
            "mapped_category": dict_udrt_kakao_cdh_updated2["mapped_category"]
        })
        print(f"{len(new_documents)}개의 문서 생성됨")


# 최종 데이터를 new_collection에 삽입
new_collection=db["udrt_kakao_cdh_old&new_place_category"]
if new_documents:
    new_collection.insert_many(new_documents)

print(f"{len(new_documents)}개의 문서를 new_collection에 삽입 완료.")